In [ ]:
################################
####### Creating workers #######
################################
using Distributed
n_workers = 10
nprocs()==1 || rmprocs(workers()); addprocs(n_workers, exeflags="--project");

In [ ]:
################################
#### Activating the project ####
################################
# @everywhere using Pkg
# @everywhere Pkg.instantiate();
# @everywhere Pkg.activate("../DistributedOptimization");
include("./src/DistributedOptimization.jl")

### Experiment setting

In [ ]:
################################
###### Problem parameters ######
################################

using DistributedArrays
@everywhere using Random
@everywhere Random.seed!(42)

n = 1000
m = 20
λ = 0.01
zero_columns = 1

# active=0 returns 1
# active=1 Returns vector of ones with zeros at the ith block of size nworkers()
# active=2 Returns vector of zeros with ones at the ith block of size nworkers()
@everywhere function zeros_at(i,active=0)
    active==0 && return 1
    to_zero = [(i-1)*(m÷nworkers()) + k for k in 1:m÷nworkers()]
    active==1 && return [all(j .!= to_zero) for j in 1:m]
    active==2 && return [any(j .== to_zero) for j in 1:m]
end
    
A = [@spawnat worker rand(n÷nworkers(),m).*zeros_at(i,zero_columns)' for (i, worker) in enumerate(workers())]
A = reshape(A, :, 1)
A = DArray(A);

x_gen = rand(m,1) .+ 10

b = [@spawnat worker A.localpart*x_opt + 0.01*rand(Poisson(1),(n÷nworkers(),1)) for worker in workers()]
b = reshape(b, :, 1)
b = DArray(b);

objective = Objective(A,b,λ); # initializes worker_objective

In [ ]:
################################
###### Learning parameters #####
################################
epochs = 0
time = 10
slow_workers = Dict(zip(workers()[1:2], [50,100]));
# slow_workers = Dict();
γ_paper = 1/objective.L
γ_piag  = 0.1/objective.L

### Experiment

In [ ]:
##############################################################
################## Enforcing JIT compilation #################
##############################################################
async_optimize(rand(m,1), PaperSolver(objective, 1/objective.L), epochs=2);
async_optimize(rand(m,1), PiagSolver(objective, 1/objective.L), epochs=2);
sync_optimize(rand(m,1), PaperSolver(objective, 1/objective.L), epochs=2);

In [ ]:
##############################################################
####################### Finding x_star #######################
##############################################################
paper_solver = PaperSolver(objective, 1/objective.L)
history_sync_paper = sync_optimize(rand(m,1), paper_solver; epochs=1000, verbose=100);
x_star = history_sync_paper.logs["x"][end];
norm(x_star-x_opt)

In [ ]:
###############################################################
######################### Experiment ##########################
###############################################################

histories = Dict{String, Vector{History}}("piag"=>[], "paper"=>[], "sync"=>[])
# histories = Dict{String, Vector{History}}("paper"=>[],"sync"=>[])

for k in 1:1
    print("◀︎"); x_init = rand(m,1)
    
    ############################# SYNC ############################
    paper_solver = PaperSolver(objective, γ_paper);
    history = sync_optimize(x_init, paper_solver; slow_workers=slow_workers, epochs=epochs, time=time, verbose=0);
    append!(histories["sync"],[history])
    print("■")
    
    ########################### PAPER ###########################
    paper_solver = PaperSolver(objective, γ_paper); # initializes worker_solver
    history = async_optimize(x_init, paper_solver; slow_workers=slow_workers, epochs=epochs, time=time, verbose=0);
    append!(histories["paper"],[history])
    print("■")
    
    ############################ PIAG #############################
    piag_solver = PiagSolver(objective, γ_piag); # initializes worker_solver
    history = async_optimize(x_init, piag_solver; slow_workers=slow_workers, epochs=epochs, time=time, verbose=0);
    append!(histories["piag"],[history])
    print("■")
    
    print("▶︎")
end
println("◇")

### Saving & Loading experiment data

In [ ]:
###############################################################
########################### Saving ############################
###############################################################

using JLD2, FileIO

file_name = "file_name"

isdir("./data") || mkdir("./data");
parameters = Dict("n"=>n,"m"=>m,"λ"=>λ,"zero_columns"=>zero_columns, "epochs"=>epochs, "x_gen"=>x_gen,
                  "time"=>time, "slow_workers"=>slow_workers, "γ_paper"=>γ_paper, "γ_piag" =>γ_piag)
save("./data/$file_name.jld2", "parameters", parameters, "histories", histories)

-- *Load from here* --

In [ ]:
##############################################################
######################### Loading ############################
##############################################################

# include("./src/DistributedOptimization.jl")
# using JLD2, FileIO

# file_name = "file_name"
# loaded_data = load("./data/$file_name.jld2")
# histories   = loaded_data["histories"]
# parameters  = loaded_data["parameters"];

# ######## Assigning dictionary values to variables ############
# for (key, value) in parameters
#     @eval ($(Symbol(key)) = $value)
# end

### Plots

In [ ]:
###############################################################
########################### Plots #############################
###############################################################

using Plots
using PlotlyJS
using PGFPlotsX
using LaTeXStrings

function learning_curves(;xlabel, ylabel, yscale, histories, file_name="", save_as="", x_star=nothing)

    save_as in ["tex", "pdf", "tikz"] ? pgfplotsx() : gr()
    
    plt = Plots.plot(yscale=yscale, legend=:topright, grid=true, linewidth=4)
    
    h(x)  = x⋅log.(x)
    ∇h(x) = log.(x) .+ 1
    
    ylabel=="norm"        && (D = (x,y) -> norm(x-y,2); ylabel!(L"||x^*-x^k||_2"))
    ylabel=="bregman"     && (D = (x,y) -> h(x) - h(y) - ∇h(y)⋅(x-y); ylabel!(L"D_h(x^*,x^k)"))
    ylabel=="f"           && ylabel!(L"f(x^k)")
    ylabel=="∇f"          && ylabel!(L"||∇f(x^k)||^2")
    
    
    xlabel=="time"        && xlabel!("Time (s)")
    xlabel=="epochs"      && xlabel!("Epochs")
    
    isnothing(x_star) && (x_stars = [history.logs["x"][end] for history in histories["sync"]])
    
    for ((algorithm, algo_histories), linestyle, color) in zip(histories, [:solid,:dot,:dash,:dash], [:palegreen2,:deepskyblue1,:salmon1,:orange])
        
        algorithm=="sync"  && (label="Synchronous Bregman Descent")
        algorithm=="paper" && (label="Asynchronous Bregman Descent")
        algorithm=="piag"  && (label="Piag Asynchronous Bregman Descent")
        
        X_tmp = []
        Y_tmp = []
        
        end_epoch = min([history.epoch for history in algo_histories]...)
        end_time  = min([history.logs["elapsed"][end] for history in algo_histories]...)
        
        for (i, history) in enumerate(algo_histories)
            if ylabel=="f"
                !haskey(history.logs, "f") && log!(history, objective.f)
                append!(Y_tmp, [history.logs["f"][1:end_epoch]])
            elseif ylabel=="∇f"
                !haskey(history.logs, "∇f") && log!(history, objective.∇f)
                append!(Y_tmp, [[norm(g,2) for g in history.logs["∇f"]][1:end_epoch]])
            elseif ylabel in ["norm", "bregman"]
                isnothing(x_star) && (x_star = x_stars[i])
                append!(Y_tmp, [[D(x_star,x) for x in history.logs["x"][1:end_epoch]]])
            end
            
            xlabel=="time" && append!(X_tmp, [history.logs["elapsed"][1:end_epoch]])
        end
        
        if xlabel=="epochs"
            X = [1:end_epoch]
            Y = mean(Y_tmp)
            V = std(Y_tmp)
        elseif xlabel=="time"
            X = mean(X_tmp)
            Y = mean(Y_tmp)[X .< end_time]
            V = std(Y_tmp)[X .< end_time]
            X = X[X .< end_time]
        end
        
        try  
            plot!(X, Y, ribbon=V, fillalpha=.3, label=label, linestyle=linestyle, color=color)
        catch e
            println(e)
            plot!(X, Y, label=label, linestyle=linestyle, color=color)
        end

    end
    
    isdir("./plots") || mkdir("./plots");
    save_as!="" && Plots.savefig(plt, "./plots/$file_name.$save_as")
    
    gr(); plt
end

In [ ]:
plt = learning_curves(xlabel    = "time", 
                      ylabel    = "f", 
                      yscale    = :log, 
                      histories = histories, 
                      file_name = file_name,
                      save_as   = "png",
                      x_star = x_gen)